In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-13 18:20:45.893367: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 18:20:45.929629: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 18:20:45.930441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 18:20:46.550881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [29]:
MAX_WORDS_IN_SENTENCE = 700
WORD2VEC_DIMENSIONS = 100

In [30]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [31]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index:
            sentence_vectors.append(embeddings_index[word])
    pad_len = vector_space_dimension-len(sentence_vectors)         
    if pad_len > 0:
        padding = [np.zeros_like(sentence_vectors[0])] * pad_len
        sentence_vectors = sentence_vectors + padding
                
    return sentence_vectors

In [32]:
#

In [33]:
dataset = pd.read_csv('./data/recipe-dataset.csv')
texts = dataset['text'].astype(str)
labels = dataset['label']

In [34]:
data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [35]:
test_x = data[:1500]
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:1500]

train_x = data[1500:]
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[1500:]

In [36]:
train_x.shape, train_y.shape

((1828, 70000), (1828,))

In [37]:
test_x.shape, test_y.shape

((1500, 70000), (1500,))

In [38]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                3500050   
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3505201 (13.37 MB)
Trainable params: 350

In [49]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

Epoch 1/5


2023-09-13 18:26:52.147861: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 511840000 exceeds 10% of free system memory.


58/58 [==============================] - 4s 51ms/step - loss: 0.0086 - accuracy: 0.9989 - val_loss: 0.0108 - val_accuracy: 0.9980
Epoch 2/5
58/58 [==============================] - 2s 43ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0199 - val_accuracy: 0.9947
Epoch 3/5
58/58 [==============================] - 3s 45ms/step - loss: 6.3295e-04 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 0.9993
Epoch 4/5
58/58 [==============================] - 3s 46ms/step - loss: 0.0014 - accuracy: 0.9989 - val_loss: 0.0122 - val_accuracy: 0.9987
Epoch 5/5
58/58 [==============================] - 3s 46ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 0.0340 - val_accuracy: 0.9953


In [50]:
model.save('./data/keras-w2v-recipe.mdl')
loaded_model = models.load_model('./data/keras-w2v-recipe.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe.mdl/assets


Loaded Model Accuracy: 99.53%


In [55]:
text = '''
To encourage thoughtful and respectful conversations
'''
v = np.array(text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE))
v = v.reshape(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS,1)
t = tf.convert_to_tensor([v])

loaded_model.predict(t)

1/1 [==============================] - 0s 15ms/step


array([[1.6574802e-05]], dtype=float32)